Tässä projektissa mukaillaan "Medallion Architecture" tyylistä tietoarkkitehtuuria, jossa data tallennetaan Bronze, Silver ja Gold-tasolle. Tämä Notebook hoitaa Bronze-tason ETL-putken. Tiivistetysti Bronze-tason ETL-putket hoitavat raakadatan tallentamisen käytettävään muotoon.

Tämän notebookin tarkoituksena on alustaa DuckDB tietokanta CSV-tiedostojen pohjalta.

In [1]:
!pip install duckdb


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Alustetaan duckdb tietokanta ja luodaan Bronze_SensorData taulu
import duckdb
from functions.database import create_db_table
import os


lite = False # Muuta false, jos haluat ajaa kaikki datat kantaan, laita True, jos haluat ajaa vain Node3200 datan kantaan (Kevennetty versio)

csv_filepath = "data/files/" # Kansio, jossa kaikki tallennettavat CSV-tiedostot sijaitsee
if not os.path.exists(csv_filepath):
    os.mkdir(csv_filepath)
    print(f'hakemisto {csv_filepath} luotu')

duckdb_path = "data/duckdb.database" # Duckdb tietokannalle annettava nimi, tallentuu lokaalisti
if lite:
    table_name = "Bronze_SensorData_lite" # Tietokantataulu Lite-version datalle. (Poista kommentti, jotta voit ajaa Lite-version)
else:
    table_name = "Bronze_SensorData" # Tietokanta tauluun tulee Bronze-prefix, jotta tiedetään, että siellä on raakadataa

schema = f'''
        CREATE TABLE {table_name} (
        node_id INTEGER NOT NULL,
        timestamp TEXT,
        x INTEGER NOT NULL,
        y INTEGER NOT NULL,
        z INTEGER NOT NULL,
        q INTEGER NOT NULL
        )
        '''

create_db_table(database=duckdb_path, table_name=table_name,schema=schema) # Luodaan tietokantataulu

Bronze_SensorData luotu


In [3]:
from functions.database import insert_csv_files_into_table

# Ajetaan halutut CSV_tiedostot tietokantaan
insert_csv_files_into_table(csv_folder=csv_filepath, table_name=table_name, db_file=duckdb_path, lite=lite)

node_3200.csv is being processed, its shape is (6489294, 6)
1 / 31 tiedostoa prosessoitu 7.60 sekunnissa
node_3224.csv is being processed, its shape is (6136400, 6)
2 / 31 tiedostoa prosessoitu 14.40 sekunnissa
node_3240.csv is being processed, its shape is (6329160, 6)
3 / 31 tiedostoa prosessoitu 21.75 sekunnissa
node_42787.csv is being processed, its shape is (8, 6)
4 / 31 tiedostoa prosessoitu 21.86 sekunnissa
node_45300.csv is being processed, its shape is (1, 6)
5 / 31 tiedostoa prosessoitu 21.87 sekunnissa
node_51719.csv is being processed, its shape is (9316264, 6)
6 / 31 tiedostoa prosessoitu 32.69 sekunnissa
node_51720.csv is being processed, its shape is (216, 6)
7 / 31 tiedostoa prosessoitu 32.84 sekunnissa
node_51735.csv is being processed, its shape is (4903274, 6)
8 / 31 tiedostoa prosessoitu 38.53 sekunnissa
node_51751.csv is being processed, its shape is (7706871, 6)
9 / 31 tiedostoa prosessoitu 47.39 sekunnissa
node_51850.csv is being processed, its shape is (7479609,

In [4]:
# Jos tarvitsee tyhjentää tietokantataulu / poistaa se. Se onnistuu alla olevalla koodilla


def drop_table(table_name, db_file):
    # Connect to DuckDB
    conn = duckdb.connect(database=db_file, read_only=False)
    # Execute the SQL statement to drop the table
    conn.execute(f'DROP TABLE IF EXISTS {table_name}')
    conn.close()
# Example usage:
table_name = 'SensorData'
duckdb_path = "./data/duckdb.database"


# Poista alla oleva kommentti, jos haluat tyhjentää tietyn taulun
# drop_table(table_name, duckdb_path)

In [4]:
import duckdb

# Otetaan yhteys tietokantaan
conn = duckdb.connect(database="./data/duckdb.database")

# Katsotaan mitä tietokantatuluja meillä on tietokannassa
query = "SELECT name FROM sqlite_master WHERE type='table'"

# Ajetaan kysely
result = conn.execute(query)

# Haetaan kaikkien taulujen nimet
table_names = result.fetchall()

# Printataan nimet
for table_name in table_names:
    print(table_name[0])

# Suljetaan yhteys 
conn.close()

Bronze_SensorData
Bronze_SensorData_lite
Silver_SensorData_lite
